In [257]:
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier


golden_number = ((1 + 5 ** 0.5) / 2)-1
def golden_hash(key,m):
    return(int(m*((key*golden_number)%1)))

In [258]:
class HashTable(object):

    MINIMUM_CAPACITY = 1

    def __init__(self, capacity=MINIMUM_CAPACITY):
        self.buckets = [None for _ in range(capacity * 1)]
        self.size = 0
        self.collisions = 0
        self.new_collisions = 0 
        self.ld_collisions = {}
    
    def load_factor(self):
        load_factor = truncate(self.size/len(self.buckets),1)
        print("Loading Factor: {0} ; Collisions: {1} ; New Collisions: {2}".format(load_factor,self.collisions, self.new_collisions))
        if load_factor not in self.ld_collisions.keys():
            self.ld_collisions[load_factor] = []
            self.ld_collisions[load_factor].append(self.new_collisions)
        else:
            self.ld_collisions[load_factor].append(self.new_collisions)

    def insert(self, key, value):
        bucket = self.hash(key)
        self.new_collisions = 0 
        while self.buckets[bucket]:
            if self.buckets[bucket]['key'] == key:
                # es el mismo key? se actualiza!
                self.buckets[bucket]['value'] = value
                self.load_factor()
                return
            bucket = (bucket + 1) % len(self.buckets)
            self.collisions +=1
            self.new_collisions +=1
 
        # oh, el valor no se encuentra? Crealo!
        self.buckets[bucket] = {'key': key, 'value': value}
        self.size += 1
        self.load_factor()
        if self.size == len(self.buckets):
            self.resize()

    def get(self, key):
        bucket = self.hash(key)
        while self.buckets[bucket]:
            if (self.buckets[bucket]['key'] == key):
                return self.buckets[bucket]['value']
            bucket = (bucket + 1) % len(self.buckets)
        raise KeyError("Lo siento, no existe la llave'{0}'!".format(key))

    def hash(self, key):
        #return(golden_hash(key,len(self.buckets)))
        return (key) % len(self.buckets)

    def resize(self):
        capacity = self.size * 2
        load_factor = truncate(self.size/len(self.buckets),1)
        print("resize!! Loading Factor: {0} ; Collisions: {1} ; New Collisions: {2}".format(load_factor,self.collisions, self.new_collisions))
        print('resize!! de {0} a {1}'.format(self.size, capacity))
        if capacity >= self.MINIMUM_CAPACITY:
            old = self.buckets
            self.buckets = [None for _ in range(capacity)]
            i = 0
            for bucket in old:
                if bucket:
                    self.buckets[i] = bucket
                    i += 1
            self.rehash()

    def rehash(self):
        old = self.buckets[:]
        self.buckets = [None for _ in self.buckets]
        for bucket in old:
            # no vacio
            if not bucket:
                continue
            new_hash = self.hash(bucket['key'])
            while self.buckets[new_hash]:
                new_hash = (new_hash + 1) % len(self.buckets)
            self.buckets[new_hash] = bucket

In [259]:
table = HashTable()

table.insert(1,3)
table.insert(3,3)
table.insert(1,1)
table.insert(5,5)
table.insert(9,2)


print(table.get(3))
print(table.get(5))
print(table.get(1))

Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! de 1 a 4
Loading Factor: 0.5 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.5 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.7 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 1.0 ; Collisions: 3 ; New Collisions: 3
resize!! Loading Factor: 1.0 ; Collisions: 3 ; New Collisions: 3
resize!! de 4 a 16
3
5
1


In [260]:
table = HashTable()
for i in range(1,1000):
    table.insert(i,i)

Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! de 1 a 4
Loading Factor: 0.5 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.7 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! de 4 a 16
Loading Factor: 0.3 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.3 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.4 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.5 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.5 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.6 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.6 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.7 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.8 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.8 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.9 ; Collisions: 0 ; New Collisi

In [261]:
lds = []
for i,(k,v)  in enumerate(table.ld_collisions.items()):
    ld = k
    average = sum(v)/len(v)
    lds.append([ld, average])
lds.sort(key=lambda x : x[0])
for i in lds:
    print('Loading Factor: {0} ; Average number of collisions: {1}'.format(i[0],i[1]))

Loading Factor: 0.2 ; Average number of collisions: 0.0
Loading Factor: 0.3 ; Average number of collisions: 0.0
Loading Factor: 0.4 ; Average number of collisions: 0.0
Loading Factor: 0.5 ; Average number of collisions: 0.02158273381294964
Loading Factor: 0.6 ; Average number of collisions: 0.29411764705882354
Loading Factor: 0.7 ; Average number of collisions: 0.2846715328467153
Loading Factor: 0.8 ; Average number of collisions: 0.375
Loading Factor: 0.9 ; Average number of collisions: 3.790909090909091
Loading Factor: 1.0 ; Average number of collisions: 13.2


No hay colisiones? Ninguna? Que sucede? 
Mi codigo esta malo? o muy bueno? (no eso nunca podría suceder) 

Veamos si el resize duplicase el tamaño del array:

[1]                              1%1=0

[X, 1]                           resize

[2, 1]                           2%2=0

[X, 1, 2, X]                     resize

[X, 1, 2, 3]                     3%4=3

[4, 1, 2, 3]                     4%4=0

[X, 1, 2, 3, 4, X, X ,X ]        resize


Parece perfecto!! Pero solo si agregamos los datos de forma secuencial :( 


In [262]:
table = HashTable()
for i in range(1,1000):
    table.insert(2*i,i)

Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! de 1 a 4
Loading Factor: 0.5 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.7 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! de 4 a 16
Loading Factor: 0.3 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.3 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.4 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.5 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.5 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.6 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.6 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.7 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.8 ; Collisions: 3 ; New Collisions: 3
Loading Factor: 0.8 ; Collisions: 7 ; New Collisions: 4
Loading Factor: 0.9 ; Collisions: 11 ; New Collis

In [263]:
lds = []
for i,(k,v)  in enumerate(table.ld_collisions.items()):
    ld = k
    average = sum(v)/len(v)
    lds.append([ld, average])
lds.sort(key=lambda x : x[0])
for i in lds:
    print('Loading Factor: {0} ; Average number of collisions: {1}'.format(i[0],i[1]))

Loading Factor: 0.2 ; Average number of collisions: 0.030303030303030304
Loading Factor: 0.3 ; Average number of collisions: 0.23529411764705882
Loading Factor: 0.4 ; Average number of collisions: 0.2462686567164179
Loading Factor: 0.5 ; Average number of collisions: 0.2446043165467626
Loading Factor: 0.6 ; Average number of collisions: 0.47794117647058826
Loading Factor: 0.7 ; Average number of collisions: 0.44525547445255476
Loading Factor: 0.8 ; Average number of collisions: 0.5661764705882353
Loading Factor: 0.9 ; Average number of collisions: 3.963636363636364
Loading Factor: 1.0 ; Average number of collisions: 26.8


En el caso contrario. Siempre hay colisión :( 

Veamos:

[2]                              2%1=0

[2, X]                           resize

[2, 4]                           4%2=0

[4, X, 2, X]                     resize 

[4, X, 2, 6]                     6%4=2 // siempre es el siguiente :( 

[4, 8, 2, 3]                     8%4=0

[8, X, 2, X, 4, X, 6, X]         resize

In [264]:
import random
randomlist = []
for i in range(0,1000):
    n = random.randint(1,10000000)
    randomlist.append(n)
table = HashTable()
for i in randomlist:
    table.insert(i,i)

Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! de 1 a 4
Loading Factor: 0.5 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.7 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 1.0 ; Collisions: 2 ; New Collisions: 2
resize!! Loading Factor: 1.0 ; Collisions: 2 ; New Collisions: 2
resize!! de 4 a 16
Loading Factor: 0.3 ; Collisions: 2 ; New Collisions: 0
Loading Factor: 0.3 ; Collisions: 2 ; New Collisions: 0
Loading Factor: 0.4 ; Collisions: 3 ; New Collisions: 1
Loading Factor: 0.5 ; Collisions: 3 ; New Collisions: 0
Loading Factor: 0.5 ; Collisions: 3 ; New Collisions: 0
Loading Factor: 0.6 ; Collisions: 5 ; New Collisions: 2
Loading Factor: 0.6 ; Collisions: 9 ; New Collisions: 4
Loading Factor: 0.7 ; Collisions: 11 ; New Collisions: 2
Loading Factor: 0.8 ; Collisions: 19 ; New Collisions: 8
Loading Factor: 0.8 ; Collisions: 30 ; New Collisions: 11
Loading Factor: 0.9 ; Collisions: 36 ; New Co

In [265]:
lds = []
for i,(k,v)  in enumerate(table.ld_collisions.items()):
    ld = k
    average = sum(v)/len(v)
    lds.append([ld, average])
lds.sort(key=lambda x : x[0])
for i in lds:
    print('Loading Factor: {0} ; Average number of collisions: {1}'.format(i[0],i[1]))

Loading Factor: 0.2 ; Average number of collisions: 0.36363636363636365
Loading Factor: 0.3 ; Average number of collisions: 0.5514705882352942
Loading Factor: 0.4 ; Average number of collisions: 0.8731343283582089
Loading Factor: 0.5 ; Average number of collisions: 1.7625899280575539
Loading Factor: 0.6 ; Average number of collisions: 2.7941176470588234
Loading Factor: 0.7 ; Average number of collisions: 8.72992700729927
Loading Factor: 0.8 ; Average number of collisions: 19.941176470588236
Loading Factor: 0.9 ; Average number of collisions: 73.36936936936937
Loading Factor: 1.0 ; Average number of collisions: 49.6


In [266]:
import random
#random.gauss(mu, sigma)
randomlist = []
for i in range(0,1000):
    n = int(random.gauss(50000,200))
    randomlist.append(n)
table = HashTable()
for i in randomlist:
    table.insert(i,i)

Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! Loading Factor: 1.0 ; Collisions: 0 ; New Collisions: 0
resize!! de 1 a 4
Loading Factor: 0.5 ; Collisions: 0 ; New Collisions: 0
Loading Factor: 0.7 ; Collisions: 1 ; New Collisions: 1
Loading Factor: 1.0 ; Collisions: 3 ; New Collisions: 2
resize!! Loading Factor: 1.0 ; Collisions: 3 ; New Collisions: 2
resize!! de 4 a 16
Loading Factor: 0.3 ; Collisions: 7 ; New Collisions: 4
Loading Factor: 0.3 ; Collisions: 7 ; New Collisions: 0
Loading Factor: 0.4 ; Collisions: 7 ; New Collisions: 0
Loading Factor: 0.5 ; Collisions: 7 ; New Collisions: 0
Loading Factor: 0.5 ; Collisions: 7 ; New Collisions: 0
Loading Factor: 0.6 ; Collisions: 12 ; New Collisions: 5
Loading Factor: 0.6 ; Collisions: 21 ; New Collisions: 9
Loading Factor: 0.7 ; Collisions: 21 ; New Collisions: 0
Loading Factor: 0.8 ; Collisions: 25 ; New Collisions: 4
Loading Factor: 0.8 ; Collisions: 29 ; New Collisions: 4
Loading Factor: 0.9 ; Collisions: 32 ; New C

In [267]:
lds = []
for i,(k,v)  in enumerate(table.ld_collisions.items()):
    ld = k
    average = sum(v)/len(v)
    lds.append([ld, average])
lds.sort(key=lambda x : x[0])
for i in lds:
    print('Loading Factor: {0} ; Average number of collisions: {1}'.format(i[0],i[1]))

Loading Factor: 0.2 ; Average number of collisions: 0.07216494845360824
Loading Factor: 0.3 ; Average number of collisions: 0.09691629955947137
Loading Factor: 0.4 ; Average number of collisions: 0.0972644376899696
Loading Factor: 0.5 ; Average number of collisions: 0.2236024844720497
Loading Factor: 0.6 ; Average number of collisions: 2.357142857142857
Loading Factor: 0.7 ; Average number of collisions: 1.9333333333333333
Loading Factor: 0.8 ; Average number of collisions: 6.777777777777778
Loading Factor: 0.9 ; Average number of collisions: 23.489795918367346
Loading Factor: 1.0 ; Average number of collisions: 49.0


Lo ideal: la efectividad de la tabla de hash no debería depender de la distribución de entrada de los valores ni de  los propios valores. 

0.6180339887498949

0.5